In [57]:
import psycopg2
import pandas as pd

import re
from ipywidgets import interact, interactive, Layout, interactive_output,VBox
import ipywidgets as widgets
from IPython.display import display, HTML
from ipywidgets import HBox, Label, FloatSlider,Dropdown,IntSlider,RadioButtons, Box, Layout
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib


from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
import pickle

In [58]:
conn = psycopg2.connect( 
                        host = 'project.cgxhdwn5zb5t.us-east-1.rds.amazonaws.com',
                        port = 5432, 
                        user = 'postgres',
                        password = 'Admin123',
                        database = 'postgres')
cursor = conn.cursor()

In [59]:
DEC2FLOAT = psycopg2.extensions.new_type(
    psycopg2.extensions.DECIMAL.values,
    'DEC2FLOAT',
    lambda value, curs: float(value) if value is not None else None)
psycopg2.extensions.register_type(DEC2FLOAT)

In [60]:
cursor.execute('Select * from "ahshouseholdclass2"')
rows = cursor.fetchall()
col_names = []
for elt in cursor.description:
    col_names.append(elt[0])

df = pd.DataFrame(data=rows, columns=col_names )

First, import our data and our prebuilt model.

In [61]:
predictors = list(df[['HHAGE', 'HINCP_BIN', 'UTILAMT','UNITSIZE','NUMPEOPLE','BEDROOMS', 
        'PERPOVLVL',  'ELECAMT', 'GASAMT', 
        'TRASHAMT', 'WATERAMT', 'OMB13CBSA','STORIES', 'HHNATVTY'
       ]])

loaded_model = pickle.load(open('test_model.sav', 'rb'))

Next, establish the variables and functions required for our visualization.

In [62]:
#interval variables
IV=['HHAGE', 'UTILAMT','NUMPEOPLE','BEDROOMS','HINCP_BIN',]
#categorical variables
CV=[ 'OMB13CBSA','UNITSIZE' ]

In [63]:
# load the model from pickle and predict according to user-input
def load_model(age, utilamt, income, unitsize, number_of_ppl, bedrooms, region):

    #set the avg on non-widget inputs
    User_input=df[predictors].iloc[0]
    User_input['PERPOVLVL']= df['PERPOVLVL'].mean()
    User_input['ELECAMT']= df['ELECAMT'].mean()
    User_input['GASAMT']= df['GASAMT'].mean()
    User_input['TRASHAMT']= df['TRASHAMT'].mean()
    User_input['WATERAMT']= df['WATERAMT'].mean()
    #User_input['OMB13CBSA']= df['OMB13CBSA'].mean()
    User_input['STORIES']= df['STORIES'].mean()
    User_input['HHNATVTY']= df['HHNATVTY'].mean()
    
    
    #interval variables
    User_input['age'] = age
    User_input['utilamt']= utilamt
    User_input['num_of_ppl']= number_of_ppl
    User_input['bedrooms']= bedrooms

    #categorical variables
    User_input[income]=1
    User_input[region]=35
    User_input[unitsize]=1

    user_proba=loaded_model.predict_proba([User_input])[:,1]
    #result = loaded_model.score(X_test, y_test)
    #print(result)
     
    #show the histgram
    #plt.figure(figsize=(15,5))
    #plt.yticks([])
    #plt.ylabel("Frequency of Predicted Instances")
    #plt.hist(df_test[df_test['shock']==0]["probability of income shock"],bins, alpha=.9, label='No Shock',color="#259433")  
    #plt.hist(df_test[df_test['shock']==1]["probability of income shock"],bins,alpha=0.7, label='Shock', color="#ff4940")
    #plt.axvline(user_proba,0,3000, linewidth=1)
    #plt.xticks([user_proba,user_proba])
    #plt.legend()
    #plt.show()

    return

style={'description_width' :"initial"}
age=widgets.IntSlider(value=30, min=15,max=90,description='Age', layout=Layout(flex='1 1 auto', width='auto'),style=style, continuous_update=False)
utilamt=widgets.IntSlider(value=250, min=0,max=10500,description='Utility Amt',style=style, continuous_update=False)
number_of_ppl=widgets.IntSlider(value=1, min=1,max=20,description='Number of People',style=style, continuous_update=False)
bedrooms=widgets.IntSlider(value=0, min=0,max=10, layout=Layout(flex='1 1 auto', width='auto'),description='Bedrooms',style=style, continuous_update=False)

unitsize=widgets.Dropdown(value='unitsize',style=style, description="Unit Size",disabled=False,layout=Layout(flex='1 1 auto', width='auto'),
                       options=[("Less than 500 square feet", 'unitsize'),
                              ("500 to 749 square feet", 'unitsize'),
                              ('750 to 999 square feet','unitsize'),
                              ('1,000 to 1,499 square feet','unitsize'),
                              ('1,500 to 1,999 square feet','unitsize'),
                              ('2,000 to 2,499 square feet','unitsize'),
                              ('2,500 to 2,999 square feet','unitsize'),
                              ('3,000 to 3,999 square feet','unitsize'),
                              ('4,000 square feet or more','unitsize')])

region=widgets.Dropdown(value='region',style=style, description="Region",disabled=False,
                    options=[("Atlanta-Sandy Springs-Roswell, GA",'region'),
                            ("Baltimore-Columbia-Towson, MD",'region'),
                            ('Birmingham-Hoover, AL','region'),
                            ('Boston-Cambridge-Newton, MA-NH','region'),
                            ('Chicago-Naperville-Elgin, IL-IN-WI','region'),
                            ('Cincinnati, OH-KY-IN','region'),
                            ('Cleveland-Elyria, OH','region'),
                            ('Dallas-Fort Worth-Arlington, TX', 'region'),
                            ('Denver-Aurora-Lakewood, CO','region'),
                            ('Detroit-Warren-Dearborn, MI','region'),
                            ('Houston-The Woodlands-Sugar Land, TX','region'),
                            ('Kansas City, MO-KS','region'),
                            ('Las Vegas-Henderson-Paradise, NV','region'),
                            ('Los Angeles-Long Beach-Anaheim, CA','region'),
                            ('Memphis, TN-MS-AR','region'),
                            ('Miami-Fort Lauderdale-West Palm Beach, FL','region'),
                            ('Milwaukee-Waukesha-West Allis, WI','region'),
                            ('Minneapolis-St. Paul-Bloomington, MN-WI','region'),
                            ('New Orleans-Metairie, LA','region'),
                            ('New York-Newark-Jersey City, NY-NJ-PA','region'),
                            ('Oklahoma City, OK','region'),
                            ('Philadelphia-Camden-Wilmington, PA-NJ-DE-MD','region'),
                            ('Phoenix-Mesa-Scottsdale, AZ', 'region'),
                            ('Pittsburgh, PA','region'),
                            ('Portland-Vancouver-Hillsboro, OR-WA','region'),
                            ('Raleigh, NC','region'),
                            ('Richmond, VA','region'),
                            ('Riverside-San Bernardino-Ontario, CA','region'),
                            ('Rochester, NY','region'),
                            ('San Antonio-New Braunfels, TX','region'),
                            ('San Francisco-Oakland-Hayward, CA','region'),
                            ('San Jose-Sunnyvale-Santa Clara, CA','region'),
                            ('Seattle-Tacoma-Bellevue, WA','region'),
                            ('Tampa-St. Petersburg-Clearwater, FL','region'),
                            ('Seattle-Tacoma-Bellevue, WA','region'),
                            ('Washington-Arlington-Alexandria, DC-VA-MD-WV','region'),
                            ('All other metropolitan areas','region'),
                            ('Not in a metropolitan area','region')])


income=widgets.Dropdown(value='income',style=style, description="Income",disabled=False,
                  options=[('(0.0, 10000.0]', 'income'),
                       ('(10000.0, 20000.0]', 'income'),
                       ('(20000.0, 30000.0]', 'income'),
                       ('(30000.0, 40000.0]', 'income'),
                       ('(40000.0, 50000.0]','income'),
                       ('(50000.0, 60000.0]', 'income'),
                       ('(60000.0, 70000.0]','income'),
                       ('(70000.0, 80000.0]',  'income' ),
                       ('(80000.0, 90000.0]', 'income'),
                       ('(90000.0, 100000.0]', 'income')])


In [64]:
left_box=VBox([age,income,number_of_ppl,region])
right_box=VBox([bedrooms,unitsize,utilamt])
ui=widgets.HBox([left_box, right_box])
output=widgets.interactive_output(load_model,{'age':age,'number_of_ppl':number_of_ppl, 'utilamt':utilamt,'bedrooms':bedrooms
                                              ,'income':income,'region':region, 'unitsize':unitsize
                                             })
output.layout.height="400px"
output.layout.width='800px'
box=widgets.VBox([ui,output])
display(box)
